In [2]:
pip install simpleai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simpleai: filename=simpleai-0.8.3-py3-none-any.whl size=100982 sha256=5b534828d18fa3b4e13ab5b679835e525adb2a9d2c2dbd06cf113ab1ffe88176
  Stored in directory: /root/.cache/pip/wheels/04/84/76/e493a82b60c25f283e4d315ac3b9b9ff44470556da775c58ad
Successfully built simpleai


1. Xếp hậu kích thước 5x5 (dùng simpleAI)
-	Dưới dạng CSP

In [8]:
from simpleai.search import CspProblem, backtrack

# Định nghĩa các biến: Q1 đến Q5 đại diện cho hàng 1 đến 5, giá trị là vị trí cột (1 đến 5)
variables = ('Q1', 'Q2', 'Q3', 'Q4', 'Q5')

# Miền giá trị: mỗi quân hậu có thể ở cột 1 đến 5
domains = {
    'Q1': [1, 2, 3, 4, 5],
    'Q2': [1, 2, 3, 4, 5],
    'Q3': [1, 2, 3, 4, 5],
    'Q4': [1, 2, 3, 4, 5],
    'Q5': [1, 2, 3, 4, 5],
}

# Ràng buộc: tất cả quân hậu phải ở các cột khác nhau
def all_different(variables, values):
    return len(values) == len(set(values))

# Ràng buộc: không có hai quân hậu nào tấn công nhau trên đường chéo
def no_diagonal_attack(variables, values):
    # Trích xuất số hàng từ tên biến (Q1 -> hàng 1, v.v.)
    row1 = int(variables[0][1:])
    row2 = int(variables[1][1:])
    col1, col2 = values
    # Kiểm tra không cùng đường chéo
    return abs(col1 - col2) != abs(row1 - row2)

# Danh sách các ràng buộc
constraints = []

# Thêm ràng buộc all_different cho tất cả các biến (các cột khác nhau)
constraints.append((variables, all_different))

# Thêm ràng buộc no_diagonal_attack cho mọi cặp quân hậu
for i in range(1, 6):
    for j in range(i + 1, 6):
        constraints.append((('Q' + str(i), 'Q' + str(j)), no_diagonal_attack))

# Tạo bài toán CSP
problem = CspProblem(variables, domains, constraints)

# Giải bằng thuật toán backtrack
solution = backtrack(problem)

# Hàm in bàn cờ
def print_board(positions):
    if positions is None:
        print("Không tìm được lời giải.")
        return
    n = 5
    # Tạo bàn cờ trống
    board = [['.' for _ in range(n)] for _ in range(n)]
    # Đặt quân hậu theo lời giải
    for var in variables:
        row = int(var[1:]) - 1  # chỉ số hàng 0-4
        col = positions[var] - 1  # chỉ số cột 0-4
        board[row][col] = 'Q'
    # In bàn cờ
    for row in board:
        print(' '.join(row))

# In lời giải
print("Lời giải:")
print(solution)
print_board(solution)


Lời giải:
{'Q1': 1, 'Q2': 3, 'Q3': 5, 'Q4': 2, 'Q5': 4}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .


2. Xếp hậu kích thước 5x5 (dùng simpleAI)
-	Ứng dụng backtrack (variable_heuristic và value_heuristic: xem cái nào tốt hơn)


In [9]:
from simpleai.search import CspProblem, backtrack, MOST_CONSTRAINED_VARIABLE, LEAST_CONSTRAINING_VALUE
import time

# Định nghĩa các biến: Q1 đến Q5 đại diện cho hàng 1 đến 5, giá trị là vị trí cột (1 đến 5)
variables = ('Q1', 'Q2', 'Q3', 'Q4', 'Q5')

# Miền giá trị: mỗi quân hậu có thể ở cột 1 đến 5
domains = {
    'Q1': [1, 2, 3, 4, 5],
    'Q2': [1, 2, 3, 4, 5],
    'Q3': [1, 2, 3, 4, 5],
    'Q4': [1, 2, 3, 4, 5],
    'Q5': [1, 2, 3, 4, 5],
}

# Ràng buộc: tất cả quân hậu phải ở các cột khác nhau
def all_different(variables, values):
    return len(values) == len(set(values))

# Ràng buộc: không có hai quân hậu nào tấn công nhau trên đường chéo
def no_diagonal_attack(variables, values):
    # Trích xuất số hàng từ tên biến (Q1 -> hàng 1, v.v.)
    row1 = int(variables[0][1:])
    row2 = int(variables[1][1:])
    col1, col2 = values
    return abs(col1 - col2) != abs(row1 - row2)

# Danh sách các ràng buộc
constraints = []

# Thêm ràng buộc all_different cho tất cả các biến (các cột khác nhau)
constraints.append((variables, all_different))

# Thêm ràng buộc no_diagonal_attack cho mọi cặp quân hậu
for i in range(1, 6):
    for j in range(i + 1, 6):
        constraints.append((('Q' + str(i), 'Q' + str(j)), no_diagonal_attack))

# Tạo bài toán CSP
problem = CspProblem(variables, domains, constraints)

# Hàm in bàn cờ
def print_board(positions):
    if positions is None:
        print("Không tìm được lời giải.")
        return
    n = 5
    board = [['.' for _ in range(n)] for _ in range(n)]
    for var in variables:
        row = int(var[1:]) - 1  # chỉ số hàng 0-4
        col = positions[var] - 1  # chỉ số cột 0-4
        board[row][col] = 'Q'
    for row in board:
        print(' '.join(row))

# Chạy với variable_heuristic = MOST_CONSTRAINED_VARIABLE (mcv), value_heuristic để trống
start_time_mcv = time.time()
solution_mcv = backtrack(problem, variable_heuristic=MOST_CONSTRAINED_VARIABLE, value_heuristic='')
time_mcv = time.time() - start_time_mcv

print("Lời giải với MOST_CONSTRAINED_VARIABLE:")
print(solution_mcv)
print_board(solution_mcv)
print(f"Thời gian thực hiện: {time_mcv} giây\n")

# Chạy với value_heuristic = LEAST_CONSTRAINING_VALUE (lvc), variable_heuristic để trống
start_time_lvc = time.time()
solution_lvc = backtrack(problem, variable_heuristic='', value_heuristic=LEAST_CONSTRAINING_VALUE)
time_lvc = time.time() - start_time_lvc

print("Lời giải với LEAST_CONSTRAINING_VALUE:")
print(solution_lvc)
print_board(solution_lvc)
print(f"Thời gian thực hiện: {time_lvc} giây\n")

# So sánh kết quả
if time_mcv < time_lvc:
    print("MOST_CONSTRAINED_VARIABLE tốt hơn (nhanh hơn).")
elif time_lvc < time_mcv:
    print("LEAST_CONSTRAINING_VALUE tốt hơn (nhanh hơn).")
else:
    print("Cả hai mất thời gian như nhau.")


Lời giải với MOST_CONSTRAINED_VARIABLE:
{'Q1': 1, 'Q2': 3, 'Q4': 2, 'Q3': 5, 'Q5': 4}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .
Thời gian thực hiện: 0.004139423370361328 giây

Lời giải với LEAST_CONSTRAINING_VALUE:
{'Q1': 1, 'Q2': 3, 'Q3': 5, 'Q4': 2, 'Q5': 4}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .
Thời gian thực hiện: 0.004885673522949219 giây

MOST_CONSTRAINED_VARIABLE tốt hơn (nhanh hơn).


3. Xếp hậu kích thước 5x5 (dùng simpleAI)
-	Áp dụng 2 chiến lược có AC-3 và ko có AC-3 backcheck (xem sử dụng AC-3 có tốt ko)


In [12]:
from simpleai.search import CspProblem, backtrack
import time

# Định nghĩa các biến: Q1 đến Q5 đại diện cho hàng 1 đến 5, giá trị là vị trí cột (1 đến 5)
variables = ('Q1', 'Q2', 'Q3', 'Q4', 'Q5')

# Miền giá trị: mỗi quân hậu có thể ở cột 1 đến 5
domains = {
    'Q1': [1, 2, 3, 4, 5],
    'Q2': [1, 2, 3, 4, 5],
    'Q3': [1, 2, 3, 4, 5],
    'Q4': [1, 2, 3, 4, 5],
    'Q5': [1, 2, 3, 4, 5],
}

# Ràng buộc: tất cả quân hậu phải ở các cột khác nhau
def all_different(variables, values):
    return len(values) == len(set(values))

# Ràng buộc: không có hai quân hậu nào tấn công nhau trên đường chéo
def no_diagonal_attack(variables, values):
    # Trích xuất số hàng từ tên biến (Q1 -> hàng 1, v.v.)
    row1 = int(variables[0][1:])
    row2 = int(variables[1][1:])
    col1, col2 = values
    return abs(col1 - col2) != abs(row1 - row2)

# Danh sách các ràng buộc
constraints = []

# Thêm ràng buộc all_different cho tất cả các biến (các cột khác nhau)
constraints.append((variables, all_different))

# Thêm ràng buộc no_diagonal_attack cho mọi cặp quân hậu
for i in range(1, 6):
    for j in range(i + 1, 6):
        constraints.append((('Q' + str(i), 'Q' + str(j)), no_diagonal_attack))

# Tạo bài toán CSP
problem = CspProblem(variables, domains, constraints)

# Hàm in bàn cờ
def print_board(positions):
    if positions is None:
        print("Không tìm được lời giải.")
        return
    n = 5
    board = [['.' for _ in range(n)] for _ in range(n)]
    for var in variables:
        row = int(var[1:]) - 1  # chỉ số hàng 0-4
        col = positions[var] - 1  # chỉ số cột 0-4
        board[row][col] = 'Q'
    for row in board:
        print(' '.join(row))

# Chạy với inference=True (sử dụng AC-3)
start_time_ac3 = time.time()
solution_ac3 = backtrack(problem, inference=True)
time_ac3 = time.time() - start_time_ac3

print("Lời giải với AC-3 (inference=True):")
print(solution_ac3)
print_board(solution_ac3)
print(f"Thời gian thực hiện: {time_ac3} giây\n")

# Chạy không sử dụng AC-3 (inference=False)
start_time_no_ac3 = time.time()
solution_no_ac3 = backtrack(problem, inference=False)
time_no_ac3 = time.time() - start_time_no_ac3

print("Lời giải không sử dụng AC-3 (inference=False):")
print(solution_no_ac3)
print_board(solution_no_ac3)
print(f"Thời gian thực hiện: {time_no_ac3} giây\n")

# So sánh kết quả
if time_ac3 < time_no_ac3:
    print("Sử dụng AC-3 tốt hơn (nhanh hơn).")
elif time_no_ac3 < time_ac3:
    print("Không sử dụng AC-3 tốt hơn (nhanh hơn).")
else:
    print("Cả hai mất thời gian như nhau.")


Lời giải với AC-3 (inference=True):
{'Q1': 1, 'Q2': 3, 'Q3': 5, 'Q4': 2, 'Q5': 4}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .
Thời gian thực hiện: 0.008074283599853516 giây

Lời giải không sử dụng AC-3 (inference=False):
{'Q1': 1, 'Q2': 3, 'Q3': 5, 'Q4': 2, 'Q5': 4}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .
Thời gian thực hiện: 0.0036661624908447266 giây

Không sử dụng AC-3 tốt hơn (nhanh hơn).


4. Xếp hậu kích thước 5x5 (dùng simpleAI)
-	Bổ sung hàm value với local search với (GA-SA-Hill climbing)


In [18]:
from simpleai.search import CspProblem, SearchProblem, hill_climbing, simulated_annealing, genetic
from simpleai.search.local import _exp_schedule
import random
import time

# Định nghĩa bài toán CSP
variables = ('Q1', 'Q2', 'Q3', 'Q4', 'Q5')
domains = {
    'Q1': [1, 2, 3, 4, 5],
    'Q2': [1, 2, 3, 4, 5],
    'Q3': [1, 2, 3, 4, 5],
    'Q4': [1, 2, 3, 4, 5],
    'Q5': [1, 2, 3, 4, 5],
}

def all_different(variables, values):
    return len(values) == len(set(values))

def no_diagonal_attack(variables, values):
    row1 = int(variables[0][1:])
    row2 = int(variables[1][1:])
    col1, col2 = values
    return abs(col1 - col2) != abs(row1 - row2)

constraints = []
constraints.append((variables, all_different))
for i in range(1, 6):
    for j in range(i + 1, 6):
        constraints.append((('Q' + str(i), 'Q' + str(j)), no_diagonal_attack))

csp_problem = CspProblem(variables, domains, constraints)

# Lớp SearchProblem tùy chỉnh cho tìm kiếm cục bộ
class NQueensProblem(SearchProblem):
    def __init__(self, n):
        self.n = n
        # Trạng thái ban đầu ngẫu nhiên: mỗi hàng có một quân hậu ở cột ngẫu nhiên
        self.initial_state = tuple(random.randint(1, n) for _ in range(n))

    def actions(self, state):
        """Tạo danh sách các hành động có thể: di chuyển quân hậu đến cột khác"""
        actions = []
        for i in range(len(state)):
            for value in range(1, self.n + 1):
                if value != state[i]:  # Chỉ di chuyển đến cột khác
                    actions.append((i, value))
        return actions

    def result(self, state, action):
        """Thực hiện hành động: di chuyển quân hậu tại hàng i đến cột value"""
        i, value = action
        new_state = list(state)
        new_state[i] = value
        return tuple(new_state)

    def value(self, state):
        """Tính giá trị trạng thái: âm số xung đột (ít xung đột = tốt hơn)"""
        conflicts = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                # Kiểm tra xung đột cột và đường chéo
                if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                    conflicts += 1
        return -conflicts  # Âm để tối đa hóa (ít xung đột hơn = tốt hơn)

    def generate_random_state(self):
        """Tạo trạng thái ngẫu nhiên cho thuật toán di truyền"""
        return tuple(random.randint(1, self.n) for _ in range(self.n))

    def crossover(self, state1, state2):
        """Lai ghép hai trạng thái cho thuật toán di truyền"""
        point = random.randint(1, len(state1) - 1)
        return state1[:point] + state2[point:]

    def mutate(self, state):
        """Đột biến trạng thái cho thuật toán di truyền"""
        i = random.randint(0, len(state) - 1)
        value = random.randint(1, self.n)
        new_state = list(state)
        new_state[i] = value
        return tuple(new_state)

# Hàm in bàn cờ
def print_board(state):
    if state is None:
        print("Không tìm được lời giải.")
        return
    n = 5
    board = [['.' for _ in range(n)] for _ in range(n)]
    for i, col in enumerate(state):
        board[i][col - 1] = 'Q'  # i là hàng, col-1 là cột (chỉ số 0-4)
    for row in board:
        print(' '.join(row))

# Khởi tạo và chạy các thuật toán tìm kiếm cục bộ
n = 5
problem = NQueensProblem(n)

# Leo đồi (Hill Climbing)
start_time_hc = time.time()
solution_hc = hill_climbing(problem)
time_hc = time.time() - start_time_hc
# Trích xuất trạng thái từ đối tượng solution (nếu có)
state_hc = solution_hc.state if solution_hc else None

print("Lời giải Leo đồi (Hill Climbing):")
print(state_hc)
print_board(state_hc)
print(f"Thời gian thực hiện: {time_hc} giây\n")

# SA (Simulated Annealing)
start_time_sa = time.time()
solution_sa = simulated_annealing(problem, schedule=_exp_schedule)
time_sa = time.time() - start_time_sa
# Trích xuất trạng thái từ đối tượng solution (nếu có)
state_sa = solution_sa.state if solution_sa else None

print("Lời giải SA (Simulated Annealing):")
print(state_sa)
print_board(state_sa)
print(f"Thời gian thực hiện: {time_sa} giây\n")

# Thuật toán di truyền (Genetic Algorithm)
start_time_ga = time.time()
solution_ga = genetic(problem, population_size=100, mutation_chance=0.1)
time_ga = time.time() - start_time_ga
# Trích xuất trạng thái từ đối tượng solution (nếu có)
state_ga = solution_ga.state if solution_ga else None

print("Lời giải GA (Genetic Algorithm):")
print(state_ga)
print_board(state_ga)
print(f"Thời gian thực hiện: {time_ga} giây\n")


Lời giải Leo đồi (Hill Climbing):
(2, 4, 1, 5, 5)
. Q . . .
. . . Q .
Q . . . .
. . . . Q
. . . . Q
Thời gian thực hiện: 0.00042557716369628906 giây

Lời giải SA (Simulated Annealing):
(2, 3, 1, 1, 3)
. Q . . .
. . Q . .
Q . . . .
Q . . . .
. . Q . .
Thời gian thực hiện: 0.00019931793212890625 giây

Lời giải GA (Genetic Algorithm):
(5, 2, 5, 1, 4)
. . . . Q
. Q . . .
. . . . Q
Q . . . .
. . . Q .
Thời gian thực hiện: 0.002236604690551758 giây

